In [1]:
import tensorflow as tf
import cv2
import numpy as np
import argparse
import csv
from contextlib import suppress
from common import estimate_pose, draw_humans, read_imgfile

import h5py
import matplotlib.pyplot as plt
import numpy as np

import time
import os
import re
from fnmatch import fnmatch

C:\Users\fragkiadakism\AppData\Local\Continuum\anaconda3\envs\tensorflow35\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def inference(imgpath, j):
    #input_width = 656
    #input_height = 368
    
    input_width = 352
    input_height = 288
    
    t0 = time.time()

    tf.reset_default_graph()
    
    from tensorflow.core.framework import graph_pb2
    graph_def = graph_pb2.GraphDef()
    # Download model from https://www.dropbox.com/s/2dw1oz9l9hi9avg/optimized_openpose.pb
    with open('models/optimized_openpose.pb', 'rb') as f:
        graph_def.ParseFromString(f.read())
    tf.import_graph_def(graph_def, name='')

    t1 = time.time()
    #print(t1 - t0)

    inputs = tf.get_default_graph().get_tensor_by_name('inputs:0')
    heatmaps_tensor = tf.get_default_graph().get_tensor_by_name('Mconv7_stage6_L2/BiasAdd:0')
    pafs_tensor = tf.get_default_graph().get_tensor_by_name('Mconv7_stage6_L1/BiasAdd:0')

    t2 = time.time()
    #print(t2 - t1)

    image = read_imgfile(imgpath, input_width, input_height)

    t3 = time.time()
    #print(t3 - t2)
    
    with tf.Session() as sess:
        heatMat, pafMat = sess.run([heatmaps_tensor, pafs_tensor], feed_dict={
            inputs: image
        })

        t4 = time.time()
        #print(t4 - t3)

        heatMat, pafMat = heatMat[0], pafMat[0]

        humans = estimate_pose(heatMat, pafMat)
        
        #trying printing coordinates for left hand, head, right hand etc. indexes: 0,2,3,4,5,6,7
        head_coco_idx = 0
        right_shoulder_idx = 2
        right_elbow_idx = 3
        right_hand_idx = 4
        left_shoulder_idx = 5
        left_elbow_idx = 6
        left_hand_idx = 7

        for human in humans:
            if human[right_hand_idx] is None:
                break
                
            head_coords = human[head_coco_idx][1]
            right_shoulder_coords = human[right_shoulder_idx][1]
            right_elbow_coords = human[right_elbow_idx][1]
            right_hand_coords = human[right_hand_idx][1]
            left_shoulder_coords = human[left_shoulder_idx][1]
            left_elbow_coords = human[left_elbow_idx][1]
            left_hand_coords = human[left_hand_idx][1]
            #print("The coordinates of the left hand are: "+ str(head_coords))
            
            fields = [head_coords, right_shoulder_coords, right_elbow_coords,right_hand_coords,  left_shoulder_coords, left_elbow_coords, left_hand_coords]
        
        #with open(r'test.csv', 'a') as f:
            #writer = csv.writer(f)
            #writer.writerow(fields)
        # end of printing
        
        # display
        image = cv2.imread(imgpath)
        image_h, image_w = image.shape[:2]
        image = draw_humans(image, humans)

        scale = 480.0 / image_h
        newh, neww = 480, int(scale * image_w + 0.5)

        image = cv2.resize(image, (neww, newh), interpolation=cv2.INTER_AREA)

        ### Uncomment below to show image with skeleton
        #cv2.imshow('result', image)
        
        ##### save image with coordinates
        #cv2.imwrite('result_'+str(j)+'.png',image)
        
        t5 = time.time()
        #print(t5 - t4)
        cv2.waitKey(0)
        
        return fields
    
    

In [3]:
# Create the dataset containing the arrays for the images and the labels
#Change w to a to append files
f = h5py.File('h5_files/test_signing.h5', 'a')

#f.create_dataset("test_set_x", (1840,7,2), np.float64)
#f.create_dataset("test_set_y", (1840,), np.int8)

In [4]:
#script to find all the jpg files and run the openpose on them
root2 = './Datasets/Test/not_signing/'
pattern = "*.jpg"
total_files = 0
total_scanned_files = 909

for root2, dirs, files in os.walk(root2, topdown=False):
    for name in files:
        total_files = total_files + 1
        if fnmatch(name, pattern):
            #print(os.path.join(root2, name))
            #Running openpose  
            with suppress(Exception):
                j = re.sub('\.jpg$', '', name)
                fpath = str(os.path.join(root2, name))    
                coord = inference(fpath, j)
                print("frame scanned: "+str(name)+"\n"+str(coord)+"\n")
                f["test_set_x"][total_scanned_files] = np.array(coord)
                f["test_set_y"][total_scanned_files] = 0
                total_scanned_files = total_scanned_files + 1
            
#print("Total files in directory"+str(total_files))
print("Total files scanned with openpose : "+str(total_scanned_files))
f.close()

frame scanned: 00001.jpg
[(0.5, 0.3055555555555556), (0.38636363636363635, 0.4722222222222222), (0.36363636363636365, 0.6666666666666666), (0.4318181818181818, 0.7777777777777778), (0.5909090909090909, 0.4722222222222222), (0.6363636363636364, 0.6388888888888888), (0.5909090909090909, 0.7777777777777778)]

frame scanned: 00002.jpg
[(0.4772727272727273, 0.3055555555555556), (0.38636363636363635, 0.4722222222222222), (0.3409090909090909, 0.6666666666666666), (0.4318181818181818, 0.7777777777777778), (0.5909090909090909, 0.4722222222222222), (0.6363636363636364, 0.6388888888888888), (0.5909090909090909, 0.75)]

frame scanned: 00003.jpg
[(0.4772727272727273, 0.3055555555555556), (0.38636363636363635, 0.4722222222222222), (0.3409090909090909, 0.6666666666666666), (0.4318181818181818, 0.8055555555555556), (0.6136363636363636, 0.4722222222222222), (0.6136363636363636, 0.6388888888888888), (0.5909090909090909, 0.7777777777777778)]

frame scanned: 00004.jpg
[(0.4772727272727273, 0.3055555555555

frame scanned: 00028.jpg
[(0.5, 0.3055555555555556), (0.38636363636363635, 0.4722222222222222), (0.3409090909090909, 0.6666666666666666), (0.4318181818181818, 0.7777777777777778), (0.6136363636363636, 0.4722222222222222), (0.6363636363636364, 0.6388888888888888), (0.5909090909090909, 0.7777777777777778)]

frame scanned: 00029.jpg
[(0.5, 0.3055555555555556), (0.38636363636363635, 0.4722222222222222), (0.3409090909090909, 0.6666666666666666), (0.4318181818181818, 0.7777777777777778), (0.6136363636363636, 0.4722222222222222), (0.6363636363636364, 0.6388888888888888), (0.5909090909090909, 0.7777777777777778)]

frame scanned: 00030.jpg
[(0.5, 0.3055555555555556), (0.38636363636363635, 0.4722222222222222), (0.3409090909090909, 0.6666666666666666), (0.4318181818181818, 0.7777777777777778), (0.6136363636363636, 0.4722222222222222), (0.6363636363636364, 0.6388888888888888), (0.5909090909090909, 0.7777777777777778)]

frame scanned: 00001.jpg
[(0.45454545454545453, 0.3055555555555556), (0.3863636

frame scanned: 00025.jpg
[(0.45454545454545453, 0.3055555555555556), (0.38636363636363635, 0.4722222222222222), (0.36363636363636365, 0.6666666666666666), (0.4090909090909091, 0.7777777777777778), (0.5909090909090909, 0.4722222222222222), (0.6136363636363636, 0.6388888888888888), (0.6363636363636364, 0.75)]

frame scanned: 00026.jpg
[(0.45454545454545453, 0.3055555555555556), (0.38636363636363635, 0.4722222222222222), (0.36363636363636365, 0.6666666666666666), (0.4090909090909091, 0.7777777777777778), (0.5909090909090909, 0.4722222222222222), (0.6136363636363636, 0.6388888888888888), (0.6363636363636364, 0.7777777777777778)]

frame scanned: 00027.jpg
[(0.45454545454545453, 0.3055555555555556), (0.38636363636363635, 0.4722222222222222), (0.36363636363636365, 0.6666666666666666), (0.4090909090909091, 0.7777777777777778), (0.5909090909090909, 0.4722222222222222), (0.6136363636363636, 0.6388888888888888), (0.6363636363636364, 0.7777777777777778)]

frame scanned: 00028.jpg
[(0.4545454545454

frame scanned: 00051.jpg
[(0.4772727272727273, 0.3055555555555556), (0.38636363636363635, 0.4722222222222222), (0.36363636363636365, 0.6666666666666666), (0.4090909090909091, 0.7777777777777778), (0.5909090909090909, 0.4722222222222222), (0.6363636363636364, 0.6388888888888888), (0.6363636363636364, 0.75)]

frame scanned: 00052.jpg
[(0.4772727272727273, 0.3333333333333333), (0.38636363636363635, 0.4722222222222222), (0.36363636363636365, 0.6666666666666666), (0.4090909090909091, 0.7777777777777778), (0.5909090909090909, 0.4722222222222222), (0.6136363636363636, 0.6388888888888888), (0.6363636363636364, 0.7777777777777778)]

frame scanned: 00053.jpg
[(0.4772727272727273, 0.3055555555555556), (0.38636363636363635, 0.4722222222222222), (0.36363636363636365, 0.6666666666666666), (0.4090909090909091, 0.7777777777777778), (0.5909090909090909, 0.4722222222222222), (0.6136363636363636, 0.6388888888888888), (0.6363636363636364, 0.7777777777777778)]

frame scanned: 00001.jpg
[(0.4772727272727273

frame scanned: 00024.jpg
[(0.4772727272727273, 0.3055555555555556), (0.38636363636363635, 0.4722222222222222), (0.36363636363636365, 0.6666666666666666), (0.4318181818181818, 0.7777777777777778), (0.5909090909090909, 0.4722222222222222), (0.6363636363636364, 0.6388888888888888), (0.6136363636363636, 0.7777777777777778)]

frame scanned: 00025.jpg
[(0.4772727272727273, 0.3055555555555556), (0.38636363636363635, 0.4722222222222222), (0.36363636363636365, 0.6666666666666666), (0.4318181818181818, 0.8055555555555556), (0.5909090909090909, 0.4722222222222222), (0.6363636363636364, 0.6388888888888888), (0.6136363636363636, 0.7777777777777778)]

frame scanned: 00026.jpg
[(0.4772727272727273, 0.3055555555555556), (0.38636363636363635, 0.4722222222222222), (0.36363636363636365, 0.6666666666666666), (0.4090909090909091, 0.7777777777777778), (0.5909090909090909, 0.4722222222222222), (0.6363636363636364, 0.6388888888888888), (0.6136363636363636, 0.7777777777777778)]

frame scanned: 00027.jpg
[(0.47

...

frame scanned: tt (20).jpg
[(0.45454545454545453, 0.3333333333333333), (0.4090909090909091, 0.4444444444444444), (0.38636363636363635, 0.5277777777777778), (0.38636363636363635, 0.5555555555555556), (0.5227272727272727, 0.4444444444444444), (0.5454545454545454, 0.5555555555555556), (0.5454545454545454, 0.6111111111111112)]

frame scanned: tt (200).jpg
[(0.5, 0.19444444444444445), (0.4090909090909091, 0.3333333333333333), (0.38636363636363635, 0.5), (0.36363636363636365, 0.5833333333333334), (0.5681818181818182, 0.3055555555555556), (0.6136363636363636, 0.5277777777777778), (0.5227272727272727, 0.5833333333333334)]

frame scanned: tt (201).jpg
[(0.5, 0.19444444444444445), (0.4090909090909091, 0.3333333333333333), (0.38636363636363635, 0.5), (0.36363636363636365, 0.5555555555555556), (0.5681818181818182, 0.3055555555555556), (0.6136363636363636, 0.5277777777777778), (0.5227272727272727, 0.5833333333333334)]

frame scanned: tt (202).jpg
[(0.4772727272727273, 0.19444444444444445), (0.40909

frame scanned: tt (23).jpg
[(0.45454545454545453, 0.3333333333333333), (0.4090909090909091, 0.4444444444444444), (0.38636363636363635, 0.5277777777777778), (0.38636363636363635, 0.5555555555555556), (0.5227272727272727, 0.4444444444444444), (0.5454545454545454, 0.5555555555555556), (0.5454545454545454, 0.6111111111111112)]

frame scanned: tt (230).jpg
[(0.5, 0.25), (0.4090909090909091, 0.3333333333333333), (0.38636363636363635, 0.4722222222222222), (0.38636363636363635, 0.5), (0.5909090909090909, 0.3333333333333333), (0.6136363636363636, 0.4722222222222222), (0.6136363636363636, 0.5555555555555556)]

frame scanned: tt (232).jpg
[(0.5, 0.25), (0.4090909090909091, 0.3333333333333333), (0.4090909090909091, 0.5277777777777778), (0.38636363636363635, 0.5), (0.5909090909090909, 0.3333333333333333), (0.6136363636363636, 0.5), (0.6136363636363636, 0.5555555555555556)]

frame scanned: tt (233).jpg
[(0.5, 0.25), (0.4090909090909091, 0.3611111111111111), (0.38636363636363635, 0.5555555555555556),

frame scanned: tt (39).jpg
[(0.4318181818181818, 0.3333333333333333), (0.38636363636363635, 0.4444444444444444), (0.36363636363636365, 0.5555555555555556), (0.36363636363636365, 0.5833333333333334), (0.5, 0.4444444444444444), (0.5454545454545454, 0.5277777777777778), (0.5227272727272727, 0.6111111111111112)]

frame scanned: tt (4).jpg
[(0.4318181818181818, 0.3333333333333333), (0.38636363636363635, 0.4444444444444444), (0.36363636363636365, 0.5555555555555556), (0.38636363636363635, 0.4722222222222222), (0.5227272727272727, 0.4444444444444444), (0.5454545454545454, 0.5555555555555556), (0.5227272727272727, 0.5833333333333334)]

frame scanned: tt (40).jpg
[(0.4318181818181818, 0.3333333333333333), (0.38636363636363635, 0.4444444444444444), (0.36363636363636365, 0.5555555555555556), (0.38636363636363635, 0.5555555555555556), (0.5, 0.4444444444444444), (0.5454545454545454, 0.5555555555555556), (0.5227272727272727, 0.6111111111111112)]

frame scanned: tt (41).jpg
[(0.4318181818181818, 0.33

frame scanned: tt (71).jpg
[(0.5227272727272727, 0.3333333333333333), (0.4318181818181818, 0.4166666666666667), (0.38636363636363635, 0.5277777777777778), (0.4318181818181818, 0.5277777777777778), (0.5681818181818182, 0.4444444444444444), (0.5909090909090909, 0.5555555555555556), (0.5227272727272727, 0.5833333333333334)]

frame scanned: tt (72).jpg
[(0.5227272727272727, 0.3333333333333333), (0.4318181818181818, 0.4166666666666667), (0.38636363636363635, 0.5277777777777778), (0.4318181818181818, 0.5277777777777778), (0.5681818181818182, 0.4444444444444444), (0.5909090909090909, 0.5277777777777778), (0.5909090909090909, 0.5833333333333334)]

frame scanned: tt (73).jpg
[(0.5227272727272727, 0.3333333333333333), (0.4318181818181818, 0.4166666666666667), (0.38636363636363635, 0.5277777777777778), (0.4090909090909091, 0.5277777777777778), (0.5681818181818182, 0.4444444444444444), (0.5909090909090909, 0.5277777777777778), (0.5681818181818182, 0.5833333333333334)]

frame scanned: tt (74).jpg
[

frame scanned: tt (96).jpg
[(0.45454545454545453, 0.3611111111111111), (0.4318181818181818, 0.4444444444444444), (0.4090909090909091, 0.5555555555555556), (0.4090909090909091, 0.6111111111111112), (0.5454545454545454, 0.4166666666666667), (0.5681818181818182, 0.5555555555555556), (0.5454545454545454, 0.5833333333333334)]

frame scanned: tt (97).jpg
[(0.4772727272727273, 0.3333333333333333), (0.4318181818181818, 0.4166666666666667), (0.4090909090909091, 0.5555555555555556), (0.45454545454545453, 0.5833333333333334), (0.5454545454545454, 0.4166666666666667), (0.5909090909090909, 0.5277777777777778), (0.5909090909090909, 0.5833333333333334)]

frame scanned: tt (98).jpg
[(0.4772727272727273, 0.3333333333333333), (0.4318181818181818, 0.4166666666666667), (0.4090909090909091, 0.5277777777777778), (0.45454545454545453, 0.5833333333333334), (0.5454545454545454, 0.4166666666666667), (0.5909090909090909, 0.5277777777777778), (0.5909090909090909, 0.5833333333333334)]

frame scanned: tt (99).jpg
[

In [ ]:
index = 6
print(f["train_set_x"][index])
print("tha label of that example is " + str(f["train_set_y"][index]))

In [ ]:
f.close()

In [ ]:
##find how many files are in a folder
root2 = './Datasets/Test/not_signing/'
pattern = "*.jpg"
total_files = 0
for root2, dirs, files in os.walk(root2, topdown=False):
    for name in files:
        if fnmatch(name, pattern):
            total_files = total_files + 1
            #print(os.path.join(root2, name))
print("Total files in directory"+str(total_files))



In [6]:
# READ THE H5 FILE
f = h5py.File('./h5_files/test_signing.h5', 'r')
dset = f["test_set_x"][:]
dset2 = f["test_set_y"][:]
index = 1654
print(dset[index])
print(dset2[index])
f.close()

[[0.45454545 0.36111111]
 [0.43181818 0.41666667]
 [0.36363636 0.55555556]
 [0.40909091 0.58333333]
 [0.54545455 0.41666667]
 [0.59090909 0.52777778]
 [0.54545455 0.58333333]]
0


In [ ]:
index = 65
print(dset[index])
print(dset2[index])
#print ("y = " + str(np.squeeze(dset2[:, index])))

f.close()